In [8]:
import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker import image_uris

role = get_execution_role()
region = boto3.Session().region_name
s3_client = boto3.client("s3")
sess = sagemaker.LocalSession()

data_bucket = "sagemaker-project-p-o3c1kiruwcnf"
train_data_prefix = "bdd100k_images/labels/manifests/manifest-scene-train.json"
val_data_prefix = "bdd100k_images/labels/manifests/manifest-scene-val.json"

output_bucket = sess.default_bucket()
output_prefix = "scene-labeling-transfer-learning"

training_image = image_uris.retrieve(region=sess.boto_region_name, framework="image-classification")

# Model Setup

In [9]:
s3_output_location = f"s3://{output_bucket}/{output_prefix}/output"

ic = sagemaker.estimator.Estimator(
    training_image,
    role,
    instance_type="local_gpu",
    instance_count=1,
    #input_mode="File",
    output_path=s3_output_location,
    sagemaker_session=sess,
)

In [10]:
ic.set_hyperparameters(
    num_layers=18,
    use_pretrained_model=1,
    image_shape="3,224,224",
    #num_classes=257,
    #num_training_samples=15420,
    mini_batch_size=8,
    epochs=2,
    learning_rate=0.01,
    precision_dtype="float32",
)

In [12]:
train_data = sagemaker.inputs.TrainingInput(
    f"s3://{data_bucket}/{train_data_prefix}/output",
    distribution='FullyReplicated',
    content_type='application/x-recordio',
    s3_data_type='AugmentedManifestFile',
    attribute_names=['source-ref'],
    input_mode='Pipe',
    record_wrapping='RecordIO'
)
validation_data = sagemaker.inputs.TrainingInput(
    f"s3://{data_bucket}/{val_data_prefix}/output",
    distribution='FullyReplicated',
    content_type='application/x-recordio',
    s3_data_type='AugmentedManifestFile',
    attribute_names=['source-ref'],
    input_mode='Pipe',
    record_wrapping='RecordIO'
)

data_channels = {"train": train_data, "validation": validation_data}

# Train

In [13]:
ic.fit(inputs=data_channels, logs=True)

Using the short-lived AWS credentials found in session. They might expire while running.


FileNotFoundError: [Errno 2] No such file or directory: 'docker': 'docker'